In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import geopandas as gpd
import fiona

import datetime

In [2]:
sheets_dict = pd.read_excel('ui_claims_county_industry.xlsx', sheet_name=None,skiprows=2,skipfooter=3)

df = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['County'] = name + 'County'
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    df = df.append(sheet)

df.reset_index(inplace=True, drop=True)

In [3]:
del df['Unnamed: 2']
del df['Unnamed: 4']

In [4]:
df = df.melt(id_vars=["County", "NAICS", "INDUSTRY NAME"], 
             var_name="Date", 
             value_name="Claims")

df['Claims'] = df['Claims'].replace(',', '', regex=True)

df['Claims'] = df['Claims'].astype(int)

In [5]:
df.head()

,County,NAICS,INDUSTRY NAME,Date,Claims
0,Appling County,11,"Agriculture, Forestry, Fishing and Hunting",03/07/2020,2
1,Appling County,21,"Mining, Quarrying, and Oil and Gas Extraction",03/07/2020,0
2,Appling County,22,Utilities,03/07/2020,0
3,Appling County,23,Construction,03/07/2020,0
4,Appling County,31-33,Manufacturing,03/07/2020,3


In [6]:
warns = pd.read_excel('warnLogs_200720.xlsx')
warns['County'] = warns['County'] + ' County'
warns['County'] = warns['County'].str.strip()

In [7]:
codes = pd.read_excel('../../data/geographies/regional-commissions.xlsx', usecols='A,B,C')
codes['County'] = codes['County'] + ' County'
codes['County'] = codes['County'].astype(str)

msa = pd.read_excel('../../data/geographies/cbsa.xlsx', sheet_name='cbsa', usecols='A,D,H,I')
msa = msa.loc[msa['State'] == 'Georgia']

In [8]:
df = pd.merge(df,codes,how='left',on='County')


In [9]:
df = pd.merge(df,msa,how='left',on='County')
warns = pd.merge(warns,codes,how='left',on='County')
warns = pd.merge(warns,msa,how='left',on='County')

In [10]:
df.head()

,County,NAICS,INDUSTRY NAME,Date,Claims,area_code,Regional Commission,CBSA Code,CBSA Title,State
0,Appling County,11,"Agriculture, Forestry, Fishing and Hunting",03/07/2020,2,13001,Heart of Georgia,NaN,NaN,NaN
1,Appling County,21,"Mining, Quarrying, and Oil and Gas Extraction",03/07/2020,0,13001,Heart of Georgia,NaN,NaN,NaN
2,Appling County,22,Utilities,03/07/2020,0,13001,Heart of Georgia,NaN,NaN,NaN
3,Appling County,23,Construction,03/07/2020,0,13001,Heart of Georgia,NaN,NaN,NaN
4,Appling County,31-33,Manufacturing,03/07/2020,3,13001,Heart of Georgia,NaN,NaN,NaN


In [11]:
today = np.datetime64('today')
lastYear = today - 365

warnsCurr = warns[(warns['Date'] > '2020-03-01') & (warns['Date'] <= today)]
warnsCurr['Period'] = 'CurrentWarns'
warnsLast = warns[(warns['Date'] > '2019-03-01') & (warns['Date'] <= lastYear)]
warnsLast['Period'] = 'Last Period'
warnPeriods = warnsCurr.append([warnsLast])

warnPeriods = warnPeriods[['ID','Company name','Est. Impact','Date','Period']]

warns = pd.merge(warns,warnPeriods,how='left',on=['ID','Company name','Est. Impact','Date'])

warns['Period'].fillna('NA',inplace=True)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
warns['State'] = 'Georgia'

In [13]:
countyWarns = warns.groupby(['County','Period']).size().reset_index()
countyWarns = countyWarns.pivot_table(0, ['County'], 'Period').reset_index(drop=False)
countyWarns.rename(columns={'County':'Area'}, inplace=True)
msaWarns = warns.groupby(['CBSA Title','Period']).size().reset_index()
msaWarns = msaWarns.pivot_table(0, ['CBSA Title'], 'Period').reset_index(drop=False)
msaWarns.rename(columns={'CBSA Title':'Area'}, inplace=True)
rcWarns = warns.groupby(['Regional Commission','Period']).size().reset_index()
rcWarns = rcWarns.pivot_table(0, ['Regional Commission'], 'Period').reset_index(drop=False)
rcWarns.rename(columns={'Regional Commission':'Area'}, inplace=True)
stateWarns = warns.groupby(['State','Period']).size().reset_index()
stateWarns = stateWarns.pivot_table(0, ['State'], 'Period').reset_index(drop=False)
stateWarns.rename(columns={'State':'Area'}, inplace=True)
del warns['State']

areaWarns = stateWarns.append([rcWarns,msaWarns,countyWarns])
areaWarns.reset_index(drop=True, inplace=True)

areaWarns.fillna(0, inplace=True)

In [14]:
areaWarns = areaWarns[['Area','CurrentWarns']]
areaWarns.fillna(0, inplace=True)

In [15]:
labor = pd.read_excel('county_laborForce.xlsx')
labor.dropna(subset=['title'], inplace=True)
labor = labor.loc[labor['Month'] == 2.0]
labor['Month'] = labor['Month'] + 1
labor['Date'] = pd.to_datetime(labor.Month.astype(int).astype(str) + '-7' + '-2020', format = '%m-%d-%Y')
labor['Date'] = labor['Date'].dt.strftime('%m/%d/%Y')
labor['title'] = labor['title'].replace(', GA', '', regex=True)

In [16]:
febLabor = labor[['title','Date','emp','unemp','laborforce']]

In [17]:
febLabor.rename(columns={'title'      : 'County',
                           'emp'        : 'FebEmployed',
                           'unemp'      : 'FebUnemployed',
                           'laborforce' : 'FebLaborForce'}, inplace=True)
febLabor.reset_index(drop=True, inplace=True)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
# create separate frame of total claims by county, not industry
countyClaims = df.groupby(['area_code','County','Date']).agg({'Claims':'sum'}).reset_index()

In [19]:
countyClaims.drop_duplicates(inplace=True)

In [20]:
countyClaims.dtypes

area_code     int64
County       object
Date         object
Claims        int64
dtype: object

In [21]:
countyClaims['Date'] =  pd.to_datetime(countyClaims['Date'], format='%m/%d/%Y')

In [22]:
countyClaims['Month'] = countyClaims['Date'].dt.strftime('%b')
countyClaims['MonthN'] = countyClaims['Date'].dt.month

In [23]:
countyClaims = countyClaims.groupby(['area_code','County','Month','MonthN']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index()

In [24]:
countyClaims.dtypes

area_code     int64
County       object
Month        object
MonthN        int64
Claims        int64
dtype: object

In [25]:
countyClaims.sort_values(['area_code','MonthN'], inplace=True)

In [26]:
del countyClaims['MonthN']

In [27]:
countyClaims.head()

,area_code,County,Month,Claims
4,13001,Appling County,Mar,159
0,13001,Appling County,Apr,1214
5,13001,Appling County,May,981
3,13001,Appling County,Jun,508
2,13001,Appling County,Jul,449


In [28]:
countyClaims.to_csv('countyMonthClaims.csv', index=False)

In [29]:
countyClaims = pd.merge(countyClaims,febLabor,how='left',on=['County','Date'])

countyClaims.FebEmployed.fillna(method='ffill', inplace=True)
countyClaims.FebUnemployed.fillna(method='ffill', inplace=True)
countyClaims.FebLaborForce.fillna(method='ffill', inplace=True)

countyClaims['CumClaims'] = countyClaims.groupby(['area_code','County'])['Claims'].apply(lambda x: x.cumsum())

KeyError: 'Date'

In [32]:
countyClaims['PctFebLabor'] = ( countyClaims.CumClaims / countyClaims.FebLaborForce ) * 100
countyClaims.PctFebLabor = countyClaims.PctFebLabor.round(1)

In [33]:
# do this for the whole state
febLaborState = febLabor.groupby('Date') \
                            .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                            .reset_index()

stateClaims = countyClaims[['County','Date','Claims']]

stateClaims = stateClaims.groupby('Date').agg({'Claims':'sum'}).reset_index()

stateClaims = pd.merge(stateClaims, febLaborState, how='left', on='Date')

stateClaims.FebEmployed.fillna(method='ffill', inplace=True)
stateClaims.FebUnemployed.fillna(method='ffill', inplace=True)
stateClaims.FebLaborForce.fillna(method='ffill', inplace=True)

In [34]:
stateClaims['State'] = 'Georgia'

In [35]:
stateClaims['CumClaims'] = stateClaims.groupby(['State'])['Claims'].apply(lambda x: x.cumsum())

In [36]:
stateClaims['PctFebLabor'] = ( stateClaims.CumClaims / stateClaims.FebLaborForce ) * 100
stateClaims.PctFebLabor = stateClaims.PctFebLabor.round(1)

In [37]:
stateClaims = stateClaims[['State','Date','Claims','FebEmployed','FebUnemployed','FebLaborForce',
                           'CumClaims','PctFebLabor']]

In [38]:
# now do the same with msa claims

In [39]:
febLaborMsa = pd.merge(febLabor, msa, how='left', on='County')

In [40]:
febLaborMsa.dropna(subset=['CBSA Code'], inplace=True)

In [41]:
febLaborMsa = febLaborMsa.groupby(['CBSA Title','Date']) \
                             .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                             .reset_index()

In [42]:
msaClaims = countyClaims[['County','Date','Claims']]
msaClaims = pd.merge(msaClaims, msa, how='left', on='County')

msaClaims.dropna(subset=['CBSA Code'], inplace=True)

msaClaims = msaClaims.groupby(['CBSA Title','Date']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index()

msaClaims = pd.merge(msaClaims,febLaborMsa,how='left',on=['CBSA Title','Date'])


msaClaims.FebEmployed.fillna(method='ffill', inplace=True)
msaClaims.FebUnemployed.fillna(method='ffill', inplace=True)
msaClaims.FebLaborForce.fillna(method='ffill', inplace=True)

msaClaims['CumClaims'] = msaClaims.groupby(['CBSA Title'])['Claims'].apply(lambda x: x.cumsum())

msaClaims['PctFebLabor'] = ( msaClaims.CumClaims / msaClaims.FebLaborForce ) * 100
msaClaims.PctFebLabor = msaClaims.PctFebLabor.round(1)

In [43]:
# now repeat with regional commissions

In [44]:
febLaborRc = pd.merge(febLabor, codes, how='left', on='County')

In [45]:
febLaborRc = febLaborRc.groupby(['Regional Commission','Date']) \
                             .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                             .reset_index()

In [46]:
rcClaims = countyClaims[['County','Date','Claims']]
rcClaims = pd.merge(rcClaims, codes, how='left', on='County')

rcClaims = rcClaims.groupby(['Regional Commission','Date']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index()

rcClaims = pd.merge(rcClaims,febLaborRc,how='left',on=['Regional Commission','Date'])


rcClaims.FebEmployed.fillna(method='ffill', inplace=True)
rcClaims.FebUnemployed.fillna(method='ffill', inplace=True)
rcClaims.FebLaborForce.fillna(method='ffill', inplace=True)

rcClaims['CumClaims'] = rcClaims.groupby(['Regional Commission'])['Claims'].apply(lambda x: x.cumsum())

rcClaims['PctFebLabor'] = ( rcClaims.CumClaims / rcClaims.FebLaborForce ) * 100
rcClaims.PctFebLabor = rcClaims.PctFebLabor.round(1)

In [47]:
del countyClaims['area_code']

msaClaims.rename(columns={'CBSA Title' : 'Area'}, inplace=True)
rcClaims.rename(columns={'Regional Commission' : 'Area'}, inplace=True)
countyClaims.rename(columns={'County' : 'Area'}, inplace=True)
stateClaims.rename(columns={'State' : 'Area'}, inplace=True)

In [48]:
# stack on top of each other and write out as a single data frame for line and bar charts

In [49]:
data = stateClaims.append([countyClaims,rcClaims,msaClaims])

In [50]:
data.reset_index(drop=True, inplace=True)

In [51]:
data = data[['Area','Date','Claims','FebLaborForce','CumClaims','PctFebLabor']]

In [52]:
dataCum = data.copy()

In [53]:
countyClaims=countyClaims.groupby('Area').apply(lambda x: x.reset_index(drop=True)).drop('Area',axis=1).reset_index()
dateMax = countyClaims.level_1.max()
countyClaims = countyClaims.loc[countyClaims['level_1'] == dateMax]
countyClaims.reset_index(drop=True, inplace=True)
countyShape = gpd.read_file("spatial/ga-counties.geojson")
countyClaims = pd.merge(countyClaims, codes, how='left', left_on='Area', right_on='County')
countyClaims.area_code = countyClaims.area_code.astype(str)

In [54]:
countyClaims = pd.merge(countyClaims,areaWarns,how='left',on='Area')

In [55]:
countyShape = pd.merge(countyShape, countyClaims, how='left', left_on='GEOID', right_on='area_code')
countyShapes = countyShape[['area_code','County','CumClaims','FebLaborForce','PctFebLabor','CurrentWarns','geometry']]

In [56]:
countyShapes = gpd.GeoDataFrame(countyShapes, geometry='geometry')
countyPoints = countyShapes.copy()
countyPoints['geometry'] = countyPoints['geometry'].centroid

In [57]:
# now reshape the industry claims data to rollup by county and industry

df['State'] = 'Georgia'

df.rename(columns={'INDUSTRY NAME':'Industry'}, inplace=True)

industryCountyClaims = df.groupby(['County','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'County':'Area'})

industryCountyClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

industryRcClaims = df.groupby(['Regional Commission','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'Regional Commission':'Area'})

industryRcClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

industryMsaClaims = df.groupby(['CBSA Title','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'CBSA Title':'Area'})

industryMsaClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)


industryStateClaims = df.groupby(['State','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'State':'Area'})

industryStateClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

del df['State']

In [58]:
# append these together and reindex them
industryClaims = industryStateClaims.append([industryCountyClaims,industryRcClaims,industryMsaClaims])

industryClaims.reset_index(drop=True, inplace=True)

industryClaims['Ind'] = np.where(industryClaims.Industry.str.contains("Accommodation"), "Food Services",
                        np.where(industryClaims.Industry.str.contains("Waste"), "Support, Waste Management",
                        np.where(industryClaims.Industry.str.contains("Agriculture"), "Agriculture",
                        np.where(industryClaims.Industry.str.contains("Entertainment"), "Entertainment",
                        np.where(industryClaims.Industry.str.contains("Construction"), "Construction",
                        np.where(industryClaims.Industry.str.contains("Educational"), "Education",
                        np.where(industryClaims.Industry.str.contains("Finance"), "Finance",
                        np.where(industryClaims.Industry.str.contains("Health"), "Health Care",
                        np.where(industryClaims.Industry.str.contains("Information"), "Information",
                        np.where(industryClaims.Industry.str.contains("Management"), "Enterprise Management",
                        np.where(industryClaims.Industry.str.contains("Manufacturing"), "Manufacturing",
                        np.where(industryClaims.Industry.str.contains("Mining"), "Oil & Gas",
                        np.where(industryClaims.Industry.str.contains("Other"), "Other",
                        np.where(industryClaims.Industry.str.contains("Technical"), "Technical Services",
                        np.where(industryClaims.Industry.str.contains("Public"), "Public Admin",
                        np.where(industryClaims.Industry.str.contains("Real Estate"), "Real Estate",
                        np.where(industryClaims.Industry.str.contains("Retail Trade"), "Retail",
                        np.where(industryClaims.Industry.str.contains("Transportation"), "Transportation",
                        np.where(industryClaims.Industry.str.contains("Unclassified"), "Unclassified",
                        np.where(industryClaims.Industry.str.contains("Utilities"), "Utilities",
                        np.where(industryClaims.Industry.str.contains("Wholesale"), "Wholesale Trade","ts")))))))))))))))))))))

In [59]:
industryClaims = industryClaims[['Area','Industry','Ind','Claims']]

In [60]:
# now get the KPIs you need in a json file
dataK = data.copy()
dataK=dataK.groupby('Area').apply(lambda x: x.reset_index(drop=True)).drop('Area',axis=1).reset_index()
dateMax = dataK.level_1.max()
dataK = dataK.loc[dataK['level_1'] == dateMax]

In [61]:
dataK.sort_values('Claims', ascending=False, inplace=True)

In [62]:
dataK.head()

,Area,level_1,Date,Claims,FebLaborForce,CumClaims,PctFebLabor
1742,Georgia,20,07/25/2020,84379,5187308.0,3226000,62.2
146,"Atlanta-Sandy Springs-Alpharetta, GA",20,07/25/2020,52528,3139975.0,2032326,64.7
20,ARC,20,07/25/2020,43242,2474606.0,1662895,67.2
1700,Fulton County,20,07/25/2020,12364,565766.0,445572,78.8
1259,DeKalb County,20,07/25/2020,7589,405645.0,271781,67.0


In [63]:
dataK = pd.merge(dataK,areaWarns,how='left',on='Area')

In [64]:
dataK = dataK[['Area','FebLaborForce','CumClaims','PctFebLabor','CurrentWarns']]

In [65]:
dataK.set_index('Area', inplace=True)

In [66]:
dataK['CurrentWarns'].fillna(0, inplace=True)
countyShapes['CurrentWarns'].fillna(0, inplace=True)
countyPoints['CurrentWarns'].fillna(0, inplace=True)

In [67]:
dataK.FebLaborForce = dataK.FebLaborForce.astype(int)
dataK.CurrentWarns = dataK.CurrentWarns.astype(int)
dataK.CumClaims = dataK.CumClaims.astype(int)

In [68]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

dataK['CumClaims'] = dataK['CumClaims'].apply(lambda x: human_format(x))
dataK['CurrentWarns'] = dataK['CurrentWarns'].apply(lambda x: human_format(x))
dataK['FebLaborForce'] = dataK['FebLaborForce'].apply(lambda x: human_format(x))

In [69]:
dataK.head()

,FebLaborForce,CumClaims,PctFebLabor,CurrentWarns
Area,,,,
Georgia,5.19M,3.23M,62.2,365
"Atlanta-Sandy Springs-Alpharetta, GA",3.14M,2.03M,64.7,217
ARC,2.47M,1.66M,67.2,190
Fulton County,566K,446K,78.8,99
DeKalb County,406K,272K,67.0,0


In [70]:
# convert kpi to strings with correct format for display purposes
dataK['PctFebLabor'] = dataK['PctFebLabor'].map('{:,.1f}'.format) + '%'

In [71]:
# now process the data file from which d3 will generate charts to only include necessary columns, etc.
data["date"] = pd.to_datetime(data["Date"]).dt.strftime('%b %d')
data = data[['Area','Date','date','Claims']]

In [72]:
industryClaims['Ind'] = np.where(industryClaims.Industry.str.contains("Accommodation"), "Food Services",
                        np.where(industryClaims.Industry.str.contains("Waste"), "Waste Management",
                        np.where(industryClaims.Industry.str.contains("Agriculture"), "Agriculture",
                        np.where(industryClaims.Industry.str.contains("Entertainment"), "Entertainment",
                        np.where(industryClaims.Industry.str.contains("Construction"), "Construction",
                        np.where(industryClaims.Industry.str.contains("Educational"), "Education",
                        np.where(industryClaims.Industry.str.contains("Finance"), "Finance",
                        np.where(industryClaims.Industry.str.contains("Health"), "Health Care",
                        np.where(industryClaims.Industry.str.contains("Information"), "Information",
                        np.where(industryClaims.Industry.str.contains("Management"), "Enterprise Management",
                        np.where(industryClaims.Industry.str.contains("Manufacturing"), "Manufacturing",
                        np.where(industryClaims.Industry.str.contains("Mining"), "Oil & Gas",
                        np.where(industryClaims.Industry.str.contains("Other"), "Other",
                        np.where(industryClaims.Industry.str.contains("Technical"), "Technical Services",
                        np.where(industryClaims.Industry.str.contains("Public"), "Public Admin",
                        np.where(industryClaims.Industry.str.contains("Real Estate"), "Real Estate",
                        np.where(industryClaims.Industry.str.contains("Retail Trade"), "Retail",
                        np.where(industryClaims.Industry.str.contains("Transportation"), "Transportation",
                        np.where(industryClaims.Industry.str.contains("Unclassified"), "Unclassified",
                        np.where(industryClaims.Industry.str.contains("Utilities"), "Utilities",
                        np.where(industryClaims.Industry.str.contains("Wholesale"), "Wholesale Trade","ts")))))))))))))))))))))

In [73]:
appWarns = warns[warns['Date'] >= '2020-01-01']

In [74]:
appWarns = appWarns[['Date','Company name','City','ZIP','County','Est. Impact','Regional Commission','CBSA Title']]
appWarns.rename(columns={'Company name' : 'Company',
                         'Regional Commissions':'RC',
                         'CBSA Title':'MSA',
                         'Est. Impact':'Employees'}, inplace=True)

In [75]:
appWarns["Date"] = pd.to_datetime(appWarns["Date"]).dt.strftime('%b %d')

In [76]:
industryRoll = industryClaims[industryClaims['Area'].str.contains("County")]
industryRoll = industryRoll[['Area','Ind','Claims']]
industryRoll = industryRoll.pivot_table('Claims', ['Area'], 'Ind')
industryRoll.reset_index(inplace=True)
industryRoll = industryRoll[~industryRoll['Area'].str.contains(",")]
codes = codes[['area_code','County']]
industryRoll.rename(columns={'Area':'County'}, inplace=True)
industryRoll = pd.merge(industryRoll,codes,how='left',on='County')
industryRoll.rename(columns={'area_code':'GEOID'}, inplace=True)

In [77]:
industryRoll['Total Claims'] = industryRoll.Agriculture + industryRoll.Construction + industryRoll.Education + \
                            industryRoll['Enterprise Management'] + industryRoll.Entertainment + industryRoll.Finance + \
                            industryRoll['Food Services'] + industryRoll['Health Care'] + industryRoll.Information + \
                            industryRoll.Manufacturing + industryRoll['Oil & Gas'] + industryRoll.Other + \
                            industryRoll['Public Admin'] + industryRoll['Real Estate'] + industryRoll.Retail + \
                            industryRoll['Technical Services'] + industryRoll.Transportation + industryRoll.Unclassified + \
                            industryRoll.Utilities + industryRoll['Waste Management'] + industryRoll['Wholesale Trade']

In [78]:
industryRoll['Other Claims'] = industryRoll['Agriculture'] + industryRoll['Oil & Gas'] + industryRoll.Utilities

In [79]:
gaDataCum = dataCum.loc[dataCum['Area'] == 'Georgia']

In [80]:
# write files
# this is the county polygon file for fill layers in mapbox
countyShapes.to_file("app-data/mapbox/countyShapes.geojson", driver='GeoJSON')
# this is the county point file for circle layers in mapbox
countyPoints.to_file("app-data/mapbox/countyPoints.geojson", driver='GeoJSON')
# this is for the github file to draw the d3 charts
data.to_csv('app-data/uiClaims.csv', index=False)
industryClaims.to_csv('app-data/industryClaims.csv', index=False)
industryRoll.to_csv('app-data/countyIndustryClaims.csv', index=False)
dataCum.to_csv('app-data/dataCum.csv', index=False)
gaDataCum.to_csv('app-data/gaDataCum.csv', index=False)
# this is for the warn logs by city for a point layer within the application
appWarns.to_csv('app-data/warns.csv', index=False)
warns.to_excel('app-data/warnsClean.xlsx', index=False)
# this is for the json object to use as dynamically generated KPIs
dataK.to_json(orient='index')

'{"Georgia":{"FebLaborForce":"5.19M","CumClaims":"3.23M","PctFebLabor":"62.2%","CurrentWarns":"365"},"Atlanta-Sandy Springs-Alpharetta, GA":{"FebLaborForce":"3.14M","CumClaims":"2.03M","PctFebLabor":"64.7%","CurrentWarns":"217"},"ARC":{"FebLaborForce":"2.47M","CumClaims":"1.66M","PctFebLabor":"67.2%","CurrentWarns":"190"},"Fulton County":{"FebLaborForce":"566K","CumClaims":"446K","PctFebLabor":"78.8%","CurrentWarns":"99"},"DeKalb County":{"FebLaborForce":"406K","CumClaims":"272K","PctFebLabor":"67.0%","CurrentWarns":"0"},"Gwinnett County":{"FebLaborForce":"497K","CumClaims":"302K","PctFebLabor":"60.9%","CurrentWarns":"23"},"Northwest":{"FebLaborForce":"427K","CumClaims":"259K","PctFebLabor":"60.6%","CurrentWarns":"18"},"Cobb County":{"FebLaborForce":"432K","CumClaims":"238K","PctFebLabor":"55.2%","CurrentWarns":"33"},"Northeast":{"FebLaborForce":"309K","CumClaims":"180K","PctFebLabor":"58.2%","CurrentWarns":"15"},"Coastal":{"FebLaborForce":"333K","CumClaims":"225K","PctFebLabor":"67.5%